# Setup The Environment

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,667 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [ ]:
from manim import *
import numpy as np


In [ ]:
class LinearRegressionExplanation(Scene):
    def construct(self):
        # Sample data points
        data_points = [(1, 2), (2, 3), (3, 5), (4, 4), (5, 6)]

        # Set up axes
        axes = Axes(
            x_range=[0, 6, 1],
            y_range=[0, 7, 1],
            axis_config={"color": BLUE},
        )
        labels = axes.get_axis_labels(x_label="x", y_label="y")

        # Plot data points
        dots = VGroup(*[Dot(point=axes.c2p(x, y), color=YELLOW) for x, y in data_points])

        # Calculate actual best-fit line
        x_values = [x for x, y in data_points]
        y_values = [y for x, y in data_points]
        n = len(data_points)
        x_mean = np.mean(x_values)
        y_mean = np.mean(y_values)
        numerator = sum((x - x_mean) * (y - y_mean) for x, y in data_points)
        denominator = sum((x - x_mean)**2 for x in x_values)
        slope = numerator / denominator
        intercept = y_mean - slope * x_mean

        # ValueTrackers for line parameters
        slope_tracker = ValueTracker(0.5)
        intercept_tracker = ValueTracker(0)

        # Best-fit line
        line = always_redraw(lambda: axes.plot(
            lambda x: slope_tracker.get_value() * x + intercept_tracker.get_value(),
            color=RED
        ))

        # Equation display
        equation = always_redraw(lambda: Tex(
            f"y = {intercept_tracker.get_value():.2f} + {slope_tracker.get_value():.2f}x",
            color=RED
        ).to_edge(UR))

        # Residuals
        residuals = always_redraw(lambda: VGroup(*[
            DashedLine(
                start=axes.c2p(x, y),
                end=axes.c2p(x, slope_tracker.get_value() * x + intercept_tracker.get_value()),
                color=GREEN,
                stroke_width=2
            ) for x, y in data_points
        ]))

        # Sum of squares text
        sum_squares_text = always_redraw(lambda: Tex(
            f"Sum of squares: {sum((y - (slope_tracker.get_value() * x + intercept_tracker.get_value()))**2 for x, y in data_points):.2f}",
            color=YELLOW
        ).next_to(equation, DOWN, aligned_edge=RIGHT))

        # Animation sequence
        self.play(Create(axes), Write(labels))
        self.play(LaggedStart(*[Create(d) for d in dots], lag_ratio=0.2))
        self.wait(1)

        # Show initial line
        self.play(Create(line), Write(equation))
        self.wait(1)

        # Add residuals
        self.play(Create(residuals))
        self.add(sum_squares_text)
        self.wait(1)

        # Animate line adjustment
        self.play(
            slope_tracker.animate.set_value(slope),
            intercept_tracker.animate.set_value(intercept),
            run_time=3,
            rate_func=smooth
        )
        self.wait(2)

        # Highlight final equation
        final_equation = Tex(
            f"y = {intercept:.2f} + {slope:.2f}x",
            color=RED
        ).to_edge(UR)
        self.play(Transform(equation, final_equation))
        self.wait(2)

        # Fade out residuals
        self.play(FadeOut(residuals), FadeOut(sum_squares_text))
        self.wait(1)

        # Final emphasize on line
        self.play(line.animate.set_stroke(width=5))
        self.wait(2)


In [ ]:
%%manim -pql -v WARNING LinearRegressionExplanation


Manim Community v0.19.0